**Configurations of 10 points and their incidence varieties: code and computations**<br>
Kelly Isham, Nathan Kaplan, Sam Kimport, Rachel Lawrence, Luke Peilen, and Max Weinreich.

This notebook is a companion to our paper *Configurations of 10 points and their incidence varieties*. All definition numbers, theorem numbers, and citation numbers match those in the paper.

In this notebook, we classify the components of incidence varieties associated to $10_3$-superfigurations up to birational equivalence, verifying Computation 3.2. More precisely, given a $10_3$-superfiguration $S$,  consider the $\mathbb{Q}$-scheme $X_S$ that parametrizes framed maps of $S$ to $\mathbb{P}^2_{\mathbb{Q}}$ (Definition 2.15). Let $(X_S)_\mathrm{red}$ be the reduced scheme associated to $X_S$; so $(X_S)_\mathrm{red}$ is a variety over $\mathbb{Q}$. We prove the following:

**Computation 3.2.** For each $10_3$-superfiguration $S$ that is not a $10_3$-configuration, there is a choice of V-shaped frame such that each $\bar{\mathbb{Q}}$-component of $(X_S)_\mathrm{red}$ is birational over ${\bar{\mathbb{Q}}}$ to one of the following:

- *a projective space $\mathbb{P}^N_{\bar{\mathbb{Q}}}$, where $0 \leq N \leq 2$*, 
    
- *a genus $1$ curve.*

To do this, we first produce a coarser list (`var_comps`) containing, for each $10_3$-superfiguration $S$ (including $10_3$-configurations), the $\mathbb{Q}$-components of $X_S$. We find: 

- 1 component of dimension $3$;

- 18 components of dimension $2$;

- 140 components of dimension $1$;

- 92 components of dimension $0$.

To work with these, we simplify the presentations of the components by iterative substitutions.

We also find more detailed information regarding the list `var_comps`. The sole component of dimension $3$ is rational, and associated to the Desargues $10_3$-configuration. The components of dimension $2$ include four K3 surfaces associated to $10_3$-configurations, studied by Sink [19]; we find that the rest are rational. The components of dimension $1$ include $11$ elliptic curves; the rest split into rational components over $\bar{\mathbb{Q}}$. This step requires transferring some computations to Magma, which has methods for determining geometric irreducible decompositions of curves. Finally, the components of dimension $0$ split into points over $\bar{\mathbb{Q}}$. We describe the field of definition of each dimension $0$ component.

To begin, we create a list of all superfigurations on 10 points. We include some methods by S. Kimport,  L. Peilen, R. Lawrence,  N. Kaplan, and M. Weinreich [15], accessible at https://rachellawrence.github.io/TenArcs/. Note that their file AllSuperfigs contains all of the superfigurations on at most 10 points; however, their point labels do not allow for a standard framed map assignment as in Definition 2.15. Therefore we start with their list of linear spaces and create our own list of superfigurations. To maintain consistency with [15], we then reindex our list.

In [1]:
linearSpaceList = load('allLinearSpaces.sobj')
AllSuperfigs = load('AllSuperfigs.sobj') 

def ConfigToGraph (g):
    maxPoint = 0
    for line in g:
        for point in line:
            if maxPoint < point:
                maxPoint = point;
    dic = {}
    for x in range(len(g)):
        dic.update({ x+1 + maxPoint : g[x] })
    graph = Graph(dic)
    bgraph = BipartiteGraph( graph, (range(1, maxPoint + 1), range(maxPoint + 1, maxPoint + len(g) + 1)))
    bgraph.name( g[0])
    return bgraph

LeviGraphList = [ConfigToGraph(g) for g in AllSuperfigs]

def is_isomorphic_as_bipartite_graphs(g1, g2): 
    g1 = g1.copy()
    g2 = g2.copy()
    for g in [g1, g2]:
        g.add_vertex(100, left=true)
        for v in g.right:
            if v!= 100:
                g.add_edge(v, 100)
    is_isomorphic = g1.is_isomorphic(g2)
    g1.delete_vertex(100)
    g2.delete_vertex(100)
    return is_isomorphic 

def bipartite_inc_graph (linear_space):
    b = len(linear_space.blocks())
    p = len(linear_space.ground_set())
    bg = BipartiteGraph();
    for v in linear_space.ground_set():
        bg.add_vertex(v, left=true)
    for e in range(p, p + len(linear_space.blocks())):
        bg.add_vertex(e, right=true)
        for v in linear_space.blocks()[e-p]:
            bg.add_edge(v,e)
    return bg


def get_index_as_superfiguration (linear_space):
    for s in range(len(AllSuperfigs)):
        if is_isomorphic_as_bipartite_graphs(LeviGraphList[s], bipartite_inc_graph(linear_space)):
            return s
        
def is_superfig(lsf, numPoints):
    for v in range(0, numPoints):
        if lsf.degree(v) <= 2:
            return false
        for b in lsf.blocks():
            if len(b) <=2:
                return false
    return true

def find_superfigs(numPoints):
    all_superfigs = []
    flag = True
    for i in range(len(linearSpaceList[numPoints-1])):
        if is_superfig(linearSpaceList[numPoints-1][i], numPoints):
            all_superfigs.append(linearSpaceList[numPoints-1][i])
    return all_superfigs


Not all linear space functions are superfigurations, so we isolate those that are. This creates a list (`superfigList`) of all superfigurations on at most 10 points. Since those on 9 or fewer points were already studied [2,3,4], we focus on 10-point superfigurations. Since indexing in Sage starts at 0, our points are in $\{0, \ldots, 9\}$.


In [2]:
all_superfigs = []
for i in range(7,11):
    l = find_superfigs(i)
    for j in range(len(l)):
        all_superfigs.append(l[j])
superfigList = [0 for k in range(163)]
for j in range(len(all_superfigs)):
    ind = get_index_as_superfiguration(all_superfigs[j])
    superfigList[ind] = all_superfigs[j]

In [3]:
for l in superfigList:
    print(l.blocks())

[[0, 1, 2], [0, 3, 4], [0, 5, 6], [1, 3, 5], [1, 4, 6], [2, 3, 6], [2, 4, 5]]
[[0, 1, 2], [0, 3, 4], [0, 5, 6], [1, 3, 7], [1, 4, 5], [2, 3, 6], [2, 5, 7], [4, 6, 7]]
[[0, 1, 2], [0, 3, 6], [0, 5, 7], [1, 3, 8], [1, 4, 7], [2, 4, 6], [2, 5, 8], [3, 4, 5], [6, 7, 8]]
[[0, 1, 2], [0, 3, 4], [0, 6, 7], [1, 3, 5], [1, 4, 6], [2, 5, 7], [2, 6, 8], [3, 7, 8], [4, 5, 8]]
[[0, 1, 2], [0, 3, 4], [0, 6, 7], [1, 3, 5], [1, 4, 6], [2, 4, 8], [2, 5, 7], [3, 7, 8], [5, 6, 8]]
[[0, 1, 2], [0, 3, 4], [0, 5, 8], [0, 6, 7], [1, 3, 5], [1, 4, 7], [1, 6, 8], [2, 3, 6], [2, 4, 5], [2, 7, 8]]
[[0, 1, 2], [0, 3, 4], [0, 5, 8], [0, 6, 7], [1, 3, 5], [1, 4, 7], [1, 6, 8], [2, 3, 6], [2, 7, 8], [4, 5, 6]]
[[0, 1, 2], [0, 3, 4], [0, 5, 8], [0, 6, 7], [1, 3, 5], [1, 4, 7], [1, 6, 8], [2, 3, 6], [2, 4, 8], [2, 5, 7]]
[[0, 1, 2], [0, 3, 6], [0, 5, 7], [1, 3, 8], [1, 4, 7], [1, 5, 6], [2, 3, 7], [2, 4, 6], [2, 5, 8], [3, 4, 5], [6, 7, 8]]
[[0, 1, 2], [0, 3, 6], [0, 4, 8], [0, 5, 7], [1, 3, 8], [1, 4, 7], [1, 5, 6], 

We introduce two methods to simplify Gröbner bases. If there exists a polynomial $f(x_1, \ldots, x_n)$ in the Gröbner basis $G$ of $I$ such that $f - c_ix_i = g(x_1, \ldots, x_{i-1}, x_{i+1}, \ldots , x_n)$, then we can eliminate $x_i$ from the ideal defined by $G$. In other words, if there exists a polynomial for which a variable appears as a linear monomial term and does not appear in any other monomial term, then we can eliminate that variable. We continue doing so until there are no polynomials in the Gröbner basis with this condition.

In [4]:
def simplify_ideal(I): 
    final_polys =  [f for f in I.groebner_basis()]
    I = Ideal(I.groebner_basis())
    varis = list(I.ring().gens())
    if len(varis)==1:
        if len(final_polys)!=1:
            return("Error: the Grobner basis of I in k[x] should be principal")
        else:
            f = final_polys[0]
            if f.degree()==1:
                return(Ideal(0), [])
            else:
                return(I, varis)
    for f in final_polys:
        exp_count =vector(f.exponents()[0])

        for e in range(1,len(f.exponents())):
            exp_count+=vector(f.exponents()[e])
        for nv in range(len(exp_count)):
            if exp_count[nv] == 1:
                for e in f.exponents():
                    if e[nv]==1 and sum(e)==1: #checking that f(x1, x2, ..., xn) has a variable xn so that
                        #f - c*xn = g(x1, x2, ..., x(n-1))
                        I = I.elimination_ideal(varis[nv])
                        varis.remove(varis[nv])
                        newR = PolynomialRing(QQ, varis)
                        I = newR.ideal(I)
                        return(I, varis)
    #if we could not eliminate a variable, just return I as is
    return(I,varis)
                        
def update_ideal(I, R):
    IB = I.groebner_basis()
    if IB == [1]:
        return(I, list(R.gens()))
    else:
        Inew = I
        varis = R.gens()
        while Inew !=(0) and len(Inew.groebner_basis()) > 0 and Inew != simplify_ideal(Inew)[0]:
            
            Inew, varis = simplify_ideal(Inew)
        if Inew != (0):
            return(Inew.groebner_basis(),varis)
        else:
            return([0], varis)

For each superfiguration representative on 10 points, the points are ordered such that we have a framed superfiguration as described in Definition 2.15. We then construct $X_S$.

In [5]:
#This takes in a polynomial ring and a number of points for the superfiguration. For the case of 10-superfigurations,
#we use the polynomial ring Q[y1, y2, y3, y4, y5, z1, z2, z3, z4, z5]. This should change if k > 10.
def config_full(poly_R,num):
 
    R= poly_R
    set_verbose(0)
    Configuration = superfigList[num]
    I = Configuration.blocks()
    


    #gives the number of lines and points in our configuration
    num_lines = len(I)  
    num_points = len(superfigList[num].ground_set())


   #We start by defining the first line with 3 points in the configuration to be x=0.
    V = [vector([0,1,1]), vector([0,0,1]), vector([0,1,0]), vector([1,1,1]),  vector([1,0,0]), vector([1,y1,z1]),
    vector([1,y2,z2]),vector([1,y3,z3]), vector([1,y4,z4]),vector([1,y5,z5])]


    
    #Builds all of the determinants for all (k choose 3) sets of points on the line
    f = []
    for k in range(0,num_lines):
        if len(I[k]) == 3:
            f.append(matrix([V[I[k][0]],V[I[k][1]],V[I[k][2]]]).det())
         
        elif len(I[k])==4:
            f.append(matrix([V[I[k][0]],V[I[k][1]],V[I[k][2]]]).det())
            f.append(matrix([V[I[k][1]],V[I[k][2]],V[I[k][3]]]).det())
            f.append(matrix([V[I[k][0]],V[I[k][1]],V[I[k][3]]]).det())
            f.append(matrix([V[I[k][0]],V[I[k][2]],V[I[k][3]]]).det())
     
        elif len(I[k])==5:
            for s in Subsets(range(5), 3):
                f.append(matrix([V[I[k][s[0]]],V[I[k][s[1]]],V[I[k][s[2]]]]).det())
              
        else:
            print( "error, invalid number of blocks")
            return

    #computes the Gröbner basis.
    J = R.ideal(f);
    GB = J.groebner_basis();
    B = R.ideal(GB)
    updated_GB, varis = update_ideal(B, R)
    if updated_GB == [0]:
        return([0], varis)
    else:
        return (updated_GB.groebner_basis(), varis)

For each superfiguration $S$ on 10 points, we compute the simplified, reduced scheme $\tilde{X}_S$ over $\mathbb{Q}$ defined by the corresponding Gröbner basis. We then compute all irreducible components of $\tilde{X}_S$. For each component, we determine its dimension.

In [6]:
QR.<y1,y2,y3,y4,y5,z1,z2,z3,z4,z5> = QQ[]
var_comps = [[], [], [], []]
varis = [y1,y2,y3,y4,y5,z1,z2,z3,z4,z5]
count = [0, 0, 0, 0]
unrealizable = 0
for num in range(12,163):
    final_GB, varis = config_full(QR, num)
    if final_GB != [1]:
        if final_GB == [0]:
            count[len(varis)]+=1
            var_comps[len(varis)].append([[0], num, 0])
        else:
            string  = ""
            for i in range(len(varis)):
                string += str(varis[i])
                if i != len(varis)-1:
                    string += ","
            A= AffineSpace(len(varis), QQ, string)
            C= A.subscheme(final_GB).reduce()
            cptnum = 0
            for c in C.irreducible_components():
                cptnum+=1
                dim = c.dimension()
                if dim >=0:
                    count[dim] +=1
                    var_comps[dim].append([c, num,cptnum])
    else:
        unrealizable +=1
print( count, "number of 0-, 1-, 2-, and 3-dimensional Q-components")
print( unrealizable, "number of 10-point superfigurations for which tilde{X}_S is empty")
                                                     

[92, 140, 18, 1] number of 0-, 1-, 2-, and 3-dimensional Q-components
18 number of 10-point superfigurations for which tilde{X}_S is empty


The single dimension 3 component comes from the well-known Desargues configuration. 

In [7]:
print(var_comps[3])

[[Closed subscheme of Affine Space of dimension 5 over Rational Field defined by:
  z1*z5 - z4*z5 + z4 - z5,
  y5*z4 - y4*z5 - z4 + z5,
  y5*z1 - y4*z5 + y4 - y5 - z1 + z5, 15, 1]]


We consider the dimension 2 components. 

In [8]:
for j in var_comps[2]:
    print("Superfiguration: ", j[1], "  Component index: ", j[2] )
    if j[0] ==[0]:
        print("  isomorphic to A^2 over Q")
        print("\n")
        print("\n")
    else:
    
        updatedi, varis = update_ideal(Ideal(j[0].defining_polynomials()), QR)
        if updatedi ==[0]:
            print("  isomorphic to A^2 over Q")
            print("\n")
            print("\n")
        else:
            updatedgb = updatedi.groebner_basis()
            polylist = [];
            for f in updatedgb:
                if(f.degree() > 0):
                    polylist.append(f)
            if(len(polylist)>0):
                print(" with defining polynomials ", polylist)
            else:
                print("  isomorphic to A^2 over Q")
            print("\n")
            print("\n")

Superfiguration:  16   Component index:  1
 with defining polynomials  [z1*z4 - z1 - z4]




Superfiguration:  16   Component index:  2
  isomorphic to A^2 over Q




Superfiguration:  16   Component index:  3
 with defining polynomials  [z1*z3 - z1*z4 - z1 + z4]




Superfiguration:  16   Component index:  4
 with defining polynomials  [z1*z4 - z3*z4 - z1 + z4]




Superfiguration:  17   Component index:  1
 with defining polynomials  [y4*z3*z5 - 1/2*y4*z5^2 - 1/2*y4*z3 - 1/2*z3*z5 + 1/2*z3]




Superfiguration:  18   Component index:  1
  isomorphic to A^2 over Q




Superfiguration:  18   Component index:  2
 with defining polynomials  [y4*z3 - y4*z5 - y4 + z5]




Superfiguration:  19   Component index:  1
 with defining polynomials  [y2^2*z3*z4^2 - y2^2*z3*z4 + y2*z3^2*z4 - y2^2*z4^2 - y2*z3*z4^2 - y2*z3^2 + y2^2*z4 - y2*z3*z4 + 2*y2*z4^2 + y2*z3 - y2*z4 + z3*z4 - z4^2]




Superfiguration:  20   Component index:  1
  isomorphic to A^2 over Q




Superfiguration:  20   Component i

Since Sink [19] has studied all components of the realization spaces of $10_3$-configurations, we can ignore these. We check which configuration labels correspond to $10_3$-configurations.


In [9]:
for sf in range(len(superfigList)):
    if len(superfigList[sf].blocks())==10:
        count=0
        for bl in superfigList[sf].blocks():
            if len(bl) ==3:
                count+=1
        if count == 10 and superfigList[sf].num_points()==10:
            print(sf)

15
16
17
18
19
20
21
22
23
24


This leaves us with one dimension-2 component not corresponding to $10_3$-configurations. 
It is clear that this scheme is isomorphic to $\mathbb{A}^2$. 
  

Next we consider the components of dimension 1. For each dimension 1 component, we compute the geometric field of irreducibility $K$. If $K \neq \mathbb{Q}$, we print a minimal polynomial along with the index of the superfiguration.

In [10]:
geo_reducible_curve_indices = []

singular.lib("primdec.lib")
for c in var_comps[1]:
    if  c[0] !=[0]:
        polys = c[0].defining_polynomials()
        I = singular.ideal(polys);
        S = singular.absPrimdecGTZ(I);
        singular.setring(S);
        b = singular.eval('absolute_primes');
        if (b.splitlines()[2] != "      _[1]=a"): #the degree of the number field is at least 2  
            R.<a> = QQ[]
            poly = b.splitlines()[2].replace("_[1]=", "");
            K.<x> = NumberField(R(poly));
            geo_reducible_curve_indices += [[c[1], c[2]]]
            print("The field of definition for superfiguration ", c[1], " cpt ", c[2], " is ","\n", K)
            print("with discriminant ", K.discriminant())
            print("\n")
            print('Defining equations: ' + str(c[0]))
            print("\n")

The field of definition for superfiguration  33  cpt  2  is  
 Number Field in x with defining polynomial a^2 + 3
with discriminant  -3


Defining equations: Closed subscheme of Affine Space of dimension 3 over Rational Field defined by:
  y2*z4 - z3*z4 + z3 - 1,
  z3^2 - z3 + 1


The field of definition for superfiguration  72  cpt  3  is  
 Number Field in x with defining polynomial a^2 - a + 1
with discriminant  -3


Defining equations: Closed subscheme of Affine Space of dimension 3 over Rational Field defined by:
  z4*z5 + z3 - z4 - z5,
  z4^2 - z4 + 1,
  z3*z4 - z4 - z5 + 1,
  z3^2 - z3*z5 + z5^2 - z3 - z5 + 1


The field of definition for superfiguration  76  cpt  3  is  
 Number Field in x with defining polynomial a^2 - a - 1
with discriminant  5


Defining equations: Closed subscheme of Affine Space of dimension 2 over Rational Field defined by:
  y4^2 + 2*y4*z5 + z5^2 - y4 - z5 - 1


The field of definition for superfiguration  78  cpt  2  is  
 Number Field in x with definin

We compute the geometric components of the above five $\mathbb{Q}$-components in Magma (see the corresponding code in [1]). In each case, the scheme splits into two genus $0$ components over a degree 2 number field.

Next, we compute the genus of each geometrically irreducible component. The next cell confirms that these components have genus 0 or 1  and prints counts of the components of each type. 

In [11]:
genus_count = [0,0]
genus_comp = [[],[],[]]
triv_count=0
for c in var_comps[1]:
    if [c[1],c[2]] not in geo_reducible_curve_indices and c[0]!=[0]: 
        gen = Curve(c[0]).genus()
        if gen > 1:
            print("Warning: genus " + gen + " found")
        else:
            genus_count[gen]+=1
            genus_comp[gen].append(c)
    elif c[0]==[0]:
        triv_count+=1
print("Count for trivial components: " + str(triv_count))
for i in [0,1]:
    print("Count for genus " + str(i) + ": " + str(genus_count[i]))


Count for trivial components: 2
Count for genus 0: 122
Count for genus 1: 11


The geometric components of dimension 0 reduced schemes are points, hence rational. To summarize, we have shown that the geometric components of the reduced schemes $(X_S)_{\mathrm{red}}$ are rational or are genus 1 curves.  This completes the proof of Computation 3.2.

**Arithmetic information.** We continue with our study by providing more information on the genus 1 curves and the fields of definition of the geometrically reducible $\mathbb{Q}$-components. We print the genus 1 curves below. In Magma, we found that the Cremona labels (in order) are 14a4, 11a3, 11a3, 11a3, 15a8, 15a8, 37a1, 11a3, 11a3, 14a4, 11a3

In [12]:
for j in genus_comp[1]:
    print("Superfiguration: ", j[1], " cpt ", j[2], " with defining polynomials: ")
    print(j[0].defining_polynomials())
    print("\n")

Superfiguration:  37  cpt  1  with defining polynomials: 
(y4*z5 - 1, y4*z3 + z3^2 - z3*z5 - y4 - z3 + 1, z3^2*z5 - z3*z5^2 - z3*z5 + z3 + z5 - 1)


Superfiguration:  40  cpt  1  with defining polynomials: 
(z1^2*z3^2 - z1^2*z3 + z1*z3^2 - z1 + z3,)


Superfiguration:  44  cpt  3  with defining polynomials: 
(y4*z2 + y4 - z2 + z5 - 1, y4*z5^2 - z2*z5^2 + z5^3 - 2*y4*z5 + z2*z5 - 2*z5^2 - y4 + z5 + 1, z2^2*z5 - z2*z5^2 - z2^2 + 2*z2*z5 - z5)


Superfiguration:  45  cpt  3  with defining polynomials: 
(z2*z5 - z4*z5 - z2 + z5, z2*z4 - z4*z5 + z5 - 1, z4^2*z5 - z4*z5^2 + z5^2 - 2*z5 + 1)


Superfiguration:  75  cpt  1  with defining polynomials: 
(y4*z5 + z5^2 - y4 - z5 + 1, y4*z4 - z4^2 + z5^2 - y4 - z5 + 1, z4^2*z5 + z4*z5^2 - z5^3 - z4^2 - z4*z5 + z5^2 + z4 - z5)


Superfiguration:  79  cpt  1  with defining polynomials: 
(z3^2*z4 - z3*z4 - z4^2 + 2*z4 - 1,)


Superfiguration:  87  cpt  3  with defining polynomials: 
(z3*z5 + z4*z5 - z5^2 - z4, z3*z4 - 2*z3 + 1, z4^2*z5 - z4*z5^2 - z4^

The dimension 0 components over $\mathbb{Q}$ are geometrically unions of points. Arithmetically, their fields of definition are of interest. 

In [13]:

singular.lib("primdec.lib")
num_fields = [[],[],[],[],[]]
for c in var_comps[0]:
    if c[0] !=[0] and c[0]!=[]:
        polys = c[0].defining_polynomials()
        I = singular.ideal(polys);
        S = singular.absPrimdecGTZ(I);
        singular.setring(S);
        b = singular.eval('absolute_primes');
        if (b.splitlines()[2] != "      _[1]=a"): #the degree of the number field is at least 2  
            R.<a> = QQ[]
            poly = b.splitlines()[2].replace("_[1]=", "");
            K.<x> = NumberField(R(poly));
            num_fields[K.degree()].append(K)
            print("The field of definition for superfiguration ", c[1], " cpt ", c[2], " is ","\n", K)
            print("with discriminant ", K.discriminant())
            print("\n")
        else:
            print("The field of definition for superfiguration ", c[1], " cpt ", c[2], " is ", "Q")
            print("\n")

The field of definition for superfiguration  13  cpt  1  is  
 Number Field in x with defining polynomial 2*a^2 - 2*a + 1
with discriminant  -4


The field of definition for superfiguration  46  cpt  1  is  Q


The field of definition for superfiguration  46  cpt  2  is  
 Number Field in x with defining polynomial a^2 + 2*a + 962
with discriminant  -4


The field of definition for superfiguration  47  cpt  1  is  Q


The field of definition for superfiguration  47  cpt  2  is  
 Number Field in x with defining polynomial a^2 - a - 1
with discriminant  5


The field of definition for superfiguration  48  cpt  1  is  
 Number Field in x with defining polynomial a^3 - a - 1
with discriminant  -23


The field of definition for superfiguration  49  cpt  1  is  Q


The field of definition for superfiguration  49  cpt  2  is  
 Number Field in x with defining polynomial a^2 - 2*a + 2
with discriminant  -4


The field of definition for superfiguration  52  cpt  1  is  Q


The field of definit

Finally, we consider the number fields K to determine whether they are pairwise isomorphic.

In [14]:
final_k_list = [[],[],[num_fields[2][0]], [num_fields[3][0]], [num_fields[4][0]]]
for d in range(2,len(num_fields)):
    for k in num_fields[d]:
        flag = True
        for l in final_k_list[d]:
            if k.is_isomorphic(l):
                flag = False
        if flag:
            final_k_list[d].append(k)
for d in range(2, len(final_k_list)):
    print("The non-isomorphic number fields of degree " + str(d) + " have the following minimal polynomials:")
    for k in final_k_list[d]:
        print(k.defining_polynomial())
    print("\n")

The non-isomorphic number fields of degree 2 have the following minimal polynomials:
2*a^2 - 2*a + 1
a^2 - a - 1
a^2 - 2*a - 1
a^2 - a + 1
a^2 - 3*a + 4


The non-isomorphic number fields of degree 3 have the following minimal polynomials:
a^3 - a - 1
a^3 - 5*a^2 + 6*a - 1
a^3 - a^2 + a + 1


The non-isomorphic number fields of degree 4 have the following minimal polynomials:
a^4 - a^3 + a^2 - a + 1


